T4. Write a program to save any keyboard Character in a txt file while program window
is open and selected.

In [3]:
from tkinter import *
from tkinter import messagebox # مشخص نشد چرا در خط بالا این ماژول فراخوانی نشده است !!
from os import path
from datetime import datetime as dt
import re

'''Registration Form'''
main_form = Tk()
start_time = dt.today().strftime('%d-%B-%Y %H:%M:%S')
main_form.title("Registration Form")
file_name = f'keylogger_1.txt' # keylogger.txt will be beside current code address
no_key = 0 # to check if any key pressed or not

# dictionaries to replace for some keyboard name with its character or understandable names          
char_dict = {"grave":"`", "minus":"-","equal":"=", "backslash":"\\", "apostrophe":"'",
             "semicolon":";", "bracketleft":"[","bracketright":"]", "comma":",",
              "period":".", "slash":"/", "space":" ","exclam":"!", "at":"@", "numbersign":"#", "dollar":"$", "percent":"%",
            "asciicircum":"^", "ampersand":"&", "asterisk":"*", "parenleft":"(",
            "parenright":")", "underscore":"_", "plus":"+", "bar":"|", "asciitilde":"~",
            "less":"<", "greater":">", "question":"?", "quotedbl":"\"", "colon":":",
            "braceleft":"{", "braceright":"}"}
             
# a list to detect and remove useless key
ignor_lst = ["Prior", "Next", "Return", "Win_L", "Win_R", "Alt_L", "Alt_R",
            "App", "Control_L", "Control_R", "Shift_R", "Shift_L", "Tab",
            'Caps_Lock', 'Insert', 'Home','Num_Lock', 'End', 'Escape',
            'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'Up',
            'Down', 'Left', 'Right', 'Clear']

# a dictionary to record any key to its Entry names
ent_dict = {'name':[], 'last name':[], 'email':[], 'username':[],'password':[]}

# a dictionary to show message for any Entry or button and pattern for use in check_entries function
valid_dict = {
    'name':["2-15 Character, just alphabet letters",
            r'^[a-zA-Z]{2,15}$'],
    'last name':["2-15 Character, just alphabet letters",
            r'^[a-zA-Z]{2,15}$'],
    'email':["17-37 Character, Begin only with Alphabet, for rest digits and (._-) allowed",
            r'^[a-zA-Z][\w\.\-]{7,24}@[a-zA-Z]{4,7}\.[a-zA-Z]{3}$'],
    'username':["8-15 Character, not begin with digits, for rest digits and (_-) allowed",
            r'^[a-zA-Z][\w\-]{7,14}$'],
    'password':["at least 8 Char, at least one Special char [#?!@$%^&*-], one number and one Uppercase letter",
            r"^(?=.*?[a-z])(?=.*?[A-Z])(?=.*?[0-9])(?=.*?[#?!@$%^&*-]).{8,}$"],
    'confirm':['Click "CONFIRM" to send your info and close form!',
            r'']}

################################ FUNCTIONS #############################################
def file_check():
    '''check if key logger is exist or not'''
    global file_name
    while True: # while file name not exist , change name of it
        if path.exists(file_name):
            num = file_name[-5]
            file_name = file_name.replace(num,str(int(num)+1))
        else:
            break
            
file_check()                        
########################################################################################
def write_typing():
    '''write key log to the keyloger.txt while typing'''
    '''check interpreted key-log to write keyloger.txt while closing form'''  
    if no_key == 0:
        keylog = "NO KEY PRESSED"
    else:
        keylog = dict2txt(ent_dict)
        
    end_time = dt.today().strftime('%d-%B-%Y %H:%M:%S')         
    with open(f'{file_name}',mode='w') as txtfile: 
        txtfile.write(f'Start key-logging on: {start_time} -------------------\n'
                        f'{keylog}\n'
                        f'End key-logging on: {end_time} -------------------\n\n')
########################################################################################

def wid_name(event):
    '''retuen Widget's name in string'''
    return f'{str(event.widget).split(".")[-1]}'
########################################################################################    
entry_name = ''
ls_indx = 0 # Last index of cursor in any Entry BEFORE key press event
cur_indx = 0  # current index of cursor in any Entry AFTER key press event
def log_entry(event):
    '''Get name of Selected Entry to record any corresponding pressed key to its Entry'''
    global entry_name, ls_indx, cur_indx
    temp = wid_name(event) # name of entry
    if temp in ent_dict:
        entry_name = temp
        
    ls_indx, cur_indx = cur_indx, c_indx(event)
    on_widget(event)
########################################################################################
def c_indx(event):
    '''A function to return index of curser in Entry''' 
    try:    
        return event.widget.index(INSERT)
    except:
        pass
    
########################################################################################

def int_keylog(event):
    '''Interpret keylogger to increase its readability'''
    global ent_dict, no_key, cur_indx
    
    log_entry(event) # Update Entry name and last index of courser
    key = event.keysym # pressed key string
    no_key = 1
    key_lst = ent_dict[entry_name] # pressed key will save in corresponding list in ent_dict
    if len(key_lst) > 0:    
        if key == 'BackSpace':
            if ls_indx - cur_indx == 1:
                key_lst.pop(cur_indx)
                 
        elif key == 'Delete' and cur_indx <= (len(key_lst) - 1):
            key_lst.pop(c_indx(event))
                 
        elif key in char_dict:
            key_lst.insert(cur_indx - 1, char_dict[key])
            
        elif key.isalnum() and len(key) == 1:
            key_lst.insert(cur_indx - 1, key)
    else:
        if key in char_dict:
            key_lst.append(char_dict[key])
            
        elif key.isalnum() and len(key) == 1:
            key_lst.append(key)
            
    check_entries(event)
    write_typing()                  
########################################################################################
def dict2txt(dict):
    '''save ent_dict to string'''
    txt = ''
    for key in dict:
        if dict[key]:
            txt += f'{key}:\n'
            for i in dict[key]:
                txt += i
            txt += '\n'
    if txt == '':
        return "(( After pressing some keys, No character to show!!  ))"
    else:
        return txt
########################################################################################
def closing():
    '''while closing form, write interpreted keylog to keyloger.txt'''
    if messagebox.askokcancel("Quit", "Do you want to quit?"):
        if no_key == 0:
            write_typing()
        main_form.destroy()
########################################################################################
def  on_widget(event, txt=''):
    '''Show a message when widget is activated'''
    widget_name = wid_name(event)
    if widget_name in valid_dict:
        txt = valid_dict[widget_name][0]
        lbl_out.configure(text=txt , fg='red', font=('Comic Sans ms', 10))
########################################################################################        
def  lev_widget(event, txt=''):
    '''Clear message when mouse not hover on "CONFIRM" button'''
    widget_name = wid_name(event)
    if widget_name in valid_dict:
        lbl_out.configure(text=txt)
########################################################################################
def check_entries(event):
    '''Check if Entry values are valid or not'''
    ent = wid_name(event)
    pattern = valid_dict[ent][1]
    entry_txt = ent_dict[ent]
    if ent != 'confirm' and entry_txt:
        if re.match(pattern, ''.join(entry_txt)):
            event.widget.configure(bg='white')

        else:
            event.widget.configure(bg='red')
                 
########################################################################################    
# >>>>>>>>>>>>> Main form widgets <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
fnt1, fnt2 = ('Century', 15), ('Century', 13)
  
lbl_name = Label(text='Name', font=fnt1)
ent_n = Entry(width = 15, font=fnt2, name='name')
ent_n.bind('<Button-1>', log_entry)
ent_n.bind('<FocusIn>', log_entry) # Keyboard focus was moved to this widget, or to a child of this widget.
ent_n.focus() # focus to Name Entry By default

lbl_spac1 = Label(text='  ', font=fnt1)
lbl_spac2 = Label(text='  ', font=fnt1)
lbl_spac3 = Label(text='  ', font=fnt1)
lbl_spac4 = Label(text='  ', font=fnt1)

lbl_lname = Label(text='Last Name',font=fnt1)
ent_lname = Entry(width = 15, font=fnt2, name='last name')
ent_lname.bind('<Button-1>', log_entry)
ent_lname.bind('<FocusIn>', log_entry)

lbl_email = Label(text='Email',font=fnt1)
ent_email = Entry(width=37 ,font=fnt1, name='email')
ent_email.bind('<Button-1>', log_entry)
ent_email.bind('<FocusIn>', log_entry)

lbl_uname = Label(text='Username',font=fnt1) # username label
ent_uname = Entry(width=15 ,font=fnt2, name='username')
ent_uname.bind('<Button-1>', log_entry)
ent_uname.bind('<FocusIn>', log_entry)

lbl_pass = Label(text='Password',font=fnt1)
ent_pass = Entry(width=15 ,font=fnt2, show='*', name='password')
ent_pass.bind('<Button-1>', log_entry)
ent_pass.bind('<FocusIn>', log_entry)

gender_var = IntVar()
rbtn_ml = Radiobutton(text='Male',variable=gender_var, value=0, font=fnt2) # Radio button to determine gender (male/female)
rbtn_fml = Radiobutton(text='Female',variable=gender_var, value=1, font=fnt2) # Radio button to determine gender (male/female)

# a button to confirm and close form
btn_conf = Button(text='CONFIRM', font=fnt1, background=f'#{63:02x}{146:02x}{220:02x}',
                  name='confirm', command=closing)
btn_conf.bind('<Enter>', on_widget)
btn_conf.bind('<Leave>', lev_widget)
# a label to show some message
lbl_out = Label()

# >>>>>>>>>>>>> Main form widgets <<<<<<<<<<<<<
# >>>>>>>>>>>>> Widgets grid <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
lbl_name.grid(row=0, column=0)
ent_n.grid(row=0, column=1)
lbl_spac1.grid(row=0, column=2)
lbl_lname.grid(row=0, column=3)
ent_lname.grid(row=0, column=4)
lbl_spac2.grid(row=0, column=5)

rbtn_ml.grid(row=1, column=0)
rbtn_fml.grid(row=1, column=1)

lbl_email.grid(row=2, column=0)
ent_email.grid(row=2, column=1, columnspan=4)

lbl_uname.grid(row=3, column=0)
ent_uname.grid(row=3, column=1)
lbl_spac3.grid(row=3, column=2)
lbl_pass.grid(row=3, column=3)
ent_pass.grid(row=3, column=4)

lbl_out.grid(row=4, column=0, columnspan=5)
btn_conf.grid(row=5, column=0, columnspan=5)
lbl_spac4.grid(row=6, column=0, columnspan=5)
# >>>>>>>>>>>>> widgets grid <<<<<<<<<<<<<<<<<<
main_form.bind('<Key>',int_keylog)
main_form.bind('<Button-1>', log_entry)
'''
(https://stackoverflow.com/questions/111155/how-do-i-handle-the-window-close-event-in-tkinter)'''
main_form.protocol("WM_DELETE_WINDOW", closing)  
mainloop()

In [5]:
import datetime 
start_time = datetime.datetime.today().strftime('%d-%B-%Y %H:%M:%S')
print(start_time)


29-October-2022 23:26:15
